In [ ]:
import sys
sys.path.append('..')
from rosiellm.RosieLLM import RosieLLM
from rosiellm.RosieSSH import RosieSSH, RosieAuth
import base64
import time

### Test RosieSSH

In [ ]:
rosie_ssh = RosieSSH()
rosie_ssh.connect()

In [ ]:
print("Streaming: ", "\n", "---------")
output = rosie_ssh.execute_command('squeue', streaming=True)
print("---------", "\n", "End Streaming")
print("Output:", "\n", output)

In [ ]:
commands = ['hostname', 'hostname', 'uptime', 'uptime', 'echo "Hello World!"', 'echo "Hello World!"']
for command in commands:
    #track time
    start = time.time()
    output = rosie_ssh.execute_command(command)
    print(f"Command[{command}, time: {time.time() - start:.2f}]:", end="")
    print(output, end="\n")
    # time.sleep(1)

In [ ]:
rosie_ssh.close()

#### Test RosieAuth

In [ ]:
credentials = "username:password"
encoded_credentials = base64.b64encode(credentials.encode("utf-8")).decode("utf-8")
authorization_header = f"Basic {encoded_credentials}"

rosie_auth = RosieAuth("username", "password")
rosie_auth_header = f"Basic {rosie_auth.get_rosie_auth()}"

print("Src Authorization Header:", f'"{authorization_header}"')
print("Lib Authorization Header:", f'"{rosie_auth_header}"')

### Test vLLM Server

In [ ]:
client = RosieLLM(
    return_openai_client=True,
)

In [ ]:
# call __dict__ recursively to get all attributes
d = client.__dict__
for key in d:
    # if key is a class object, call __dict__ recursively
    if hasattr(d[key], '__dict__'):
        print(f"{key}:")
        for subkey in d[key].__dict__:
            if (subkey == "_custom_headers"):
                # hide password to avoid accidental exposure
                header_dict = d[key].__dict__[subkey]
                header_str = str(header_dict)
                idx1 = header_str.find("'Basic ")
                idx2 = header_str.find("'", idx1 + 1)
                if idx1 != -1 and idx2 != -1:
                    print(f'    {subkey}: {header_str[:idx1 + len("Basic ")]} <***>{header_str[idx2:]}')
                else:
                    print(f'    {subkey}: {header_str}')
            else:
                print(f"    {subkey}: {d[key].__dict__[subkey]}")
    else:
        print(f"{key}: {d[key]}")

In [ ]:
# client = client.http_client

In [ ]:
client.isRunning

In [ ]:
completion = client.chat.completions.create(
  model="NousResearch/Meta-Llama-3-8B-Instruct",
  messages=[
    {"role": "user", "content": "What is AI Club at MSOE?"},
  ],
  stream=True,
)

for chunk in completion:
  print(chunk.choices[0].delta.content, end="")
# print(completion.choices[0].message.content)